In [7]:
import pyautogui
import pandas as pd
import winsound
import threading
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter.scrolledtext import ScrolledText

# VARIÁVEIS CONTROLADAS PELA INTERFACE
arquivo_excel = None
col_inicio = 2
col_fim = 200
inserir_preco = False

cancelar_execucao = False
pausado = False
pause_event = threading.Event()
pause_event.set()  # começa liberado


# LOG VISUAL
def log(msg):
    txt_log.insert(tk.END, msg + "\n")
    txt_log.see(tk.END)
    root.update_idletasks()


# CONTROLE DE PAUSA / CANCELAMENTO
def checar_pausa_cancelamento():
    if cancelar_execucao:
        log("Processo cancelado pelo usuário")
        return False
    pause_event.wait()
    return True


# CÓDIGO
def executar():

    global arquivo_excel, col_inicio, col_fim, inserir_preco
    global cancelar_execucao

    if not arquivo_excel:
        messagebox.showwarning("Atenção", "Selecione o arquivo Excel.")
        return

    try:
        col_inicio = int(ent_inicio.get())
        col_fim = int(ent_fim.get())
    except:
        messagebox.showerror("Erro", "Intervalo de colunas inválido.")
        return

    inserir_preco = chk_preco_var.get()
    cancelar_execucao = False
    pause_event.set()

    log("Processo iniciado")

    # BASE TRANSPORTADORAS
    lojasGpAtualCargas = [48872,52251,9,22,23,40,54,60]
    lojasGpDiskTenha= [48872,4683,35,34]
    lojasGpModular = [48872,7076,1,2,3,5,7,11,12,13,14,15,16,17,18,19,20,24,25,26,27,37,41,42,45,46,47,48,49,50,51,55,56,58,59,61,63,64,66]
    lojasGpNativa = [48872,143114,4,6,8,10,44,65]

    lojasDensoBraspress = [6954,96058,2,3,5,7,11,15,16,17,20,22,23,40,41,42,45,46,48,49,54,56,60,61,65]
    lojasDensoExSãoMiguel = [6954,32302,1,12,13,18,34,35,50,51,53,58,59]
    lojasDensoMeridional = [6954,16346,24,25,26,27,37,47,55,63,64]
    lojasDensoRodonaves = [6954,5111,14,19]
    lojasDensoRodovitor = [6954,182891,8,9,44]
    lojasDensoTex = [6954,145140,4,6]

    transp = [
        lojasGpAtualCargas, lojasGpDiskTenha, lojasGpModular, lojasGpNativa,
        lojasDensoBraspress, lojasDensoExSãoMiguel, lojasDensoMeridional,
        lojasDensoRodonaves, lojasDensoRodovitor, lojasDensoTex
    ]

    
    # LEITURA EXCEL
    df = pd.read_excel(arquivo_excel)
    colunas = df.columns[col_inicio:col_fim]

    cln = []
    for c in range(len(colunas)):
        cln.append(colunas[c])

    for c in range(len(cln)):

        if not checar_pausa_cancelamento():
            return

        log(f"Processando Loja: {cln[c]}")
        pyautogui.sleep(6)
        planilha = pd.read_excel(arquivo_excel)
        valores_coluna = planilha['Dados'][:7]
        pyautogui.sleep(2)
        pyautogui.click(223,234)
        pyautogui.typewrite(str(cln[c]))
        
        #TOP
        pyautogui.sleep(2)
        pyautogui.click(223,413)
        pyautogui.typewrite(str(int(valores_coluna[2])))
        
        #PARCEIRO
        pyautogui.sleep(2)
        pyautogui.click(223,322)
        pyautogui.typewrite(str(int(valores_coluna[1])))

        #TIPO NEGOCIAÇÃO
        pyautogui.sleep(4)
        pyautogui.click(223,354)
        pyautogui.typewrite(str(int(valores_coluna[3])))

        #PREVISÃO DE ENTREGA
        pyautogui.sleep(2)
        pyautogui.click(226,384)
        pyautogui.typewrite(str(int(valores_coluna[5])))
        
        #COMPRADOR
        pyautogui.sleep(2)
        pyautogui.click(224,533)
        pyautogui.typewrite(str(int(valores_coluna[4])))
        
        #CONFIRMA
        pyautogui.sleep(2)
        pyautogui.click(124,147)
        pyautogui.sleep(3)
        
        #Posições de campos destinados aos códigos
        x_cod, y_cod = 817, 293
        x_qtd, y_qtd = 1169, 294
        x_valor, y_valor = 997, 292

        x_cod_lim, y_cod_lim = 815, 521
        x_qtd_lim, y_qtd_lim = 1160, 520
        x_valor_lim, y_valor_lim = 1006, 518

        repeticao = 0

        for index, row in df.iterrows():

            if not checar_pausa_cancelamento():
                return

            if repeticao < 10:

                if not pd.isnull(row[cln[c]]) and row[cln[c]] != 0 and row[cln[c]] != '':
                    
                    #Inserindo Código
                    log(f" Item código adicionado {row['Código']}")
                    pyautogui.sleep(2)
                    pyautogui.click(x_cod, y_cod)
                    pyautogui.sleep(2)
                    pyautogui.typewrite(str(row['Código']).upper())
                    pyautogui.sleep(2)
                    
                    #Inserindo Quantidade
                    pyautogui.sleep(2)
                    pyautogui.click(x_qtd, y_qtd)
                    pyautogui.sleep(2)
                    pyautogui.typewrite(str(int(row[cln[c]])))
                    
                    #Inserindo preço
                    if inserir_preco:
                        
                        pyautogui.sleep(2)
                        pyautogui.click(x_valor, y_valor)
                        pyautogui.sleep(5)
                        pyautogui.typewrite(str(float(row['Valor'])))

                    pyautogui.sleep(3)
                    
                    #Confirmar
                    pyautogui.click(1006,231)
                    pyautogui.click(1006,231)
                    
                    pyautogui.sleep(3)
                    
                    #Novo Item
                    pyautogui.click(946,227)
                    pyautogui.click(946,227)

                    y_cod += 25
                    y_qtd += 25
                    y_valor += 25
                    repeticao += 1

            else:

                if not pd.isnull(row[cln[c]]) and row[cln[c]] != 0 and row[cln[c]] != '':
                    
                    #Inserindo Código
                    log(f"Item (limite) código {row['Código']}")
                    pyautogui.sleep(2)
                    pyautogui.click(x_cod_lim, y_cod_lim)
                    pyautogui.sleep(2)
                    pyautogui.typewrite(str(row['Código']).upper())
                    pyautogui.sleep(2)
                    
                    
                    #Inserindo Quantidade
                    pyautogui.sleep(2)
                    pyautogui.click(x_qtd_lim, y_qtd_lim)
                    pyautogui.sleep(2)
                    pyautogui.typewrite(str(int(row[cln[c]])))

                    if inserir_preco:
                        
                        pyautogui.sleep(2)
                        pyautogui.click(x_valor_lim, y_valor_lim)
                        pyautogui.sleep(5)
                        pyautogui.typewrite(str(float(row['Valor'])))

                    pyautogui.sleep(3)
                    
                    #Confirmar Item
                    pyautogui.click(1006,227)
                    pyautogui.click(1006,227)
                    
                    pyautogui.sleep(3)
                    
                    #Novo Item
                    pyautogui.click(946,227)
                    pyautogui.click(946,227)
        

        pyautogui.sleep(3)
        pyautogui.click(1871,771)
        
        #Novo pedido
        pyautogui.click(61,142)
        pyautogui.click(61,142)

    winsound.Beep(1200, 600)
    log("Processo finalizado com sucesso")
    messagebox.showinfo("Finalizado", "Processo concluído com sucesso.")

# INTERFACE
def iniciar():
    threading.Thread(target=executar, daemon=True).start()

def cancelar():
    global cancelar_execucao
    cancelar_execucao = True
    pause_event.set()
    log("Cancelamento solicitado")

def selecionar_excel():
    global arquivo_excel
    arquivo_excel = filedialog.askopenfilename(
        filetypes=[("Excel", "*.xlsx *.xls")]
    )
    if arquivo_excel:
        lbl_excel.config(text=arquivo_excel.split("/")[-1])
        log(f"Arquivo selecionado: {arquivo_excel}")

root = tk.Tk()
root.title("Digitação Nacional")
root.geometry("700x560")
root.resizable(False, False)

tk.Label(root, text="Automação de Digitação", font=("Arial", 14, "bold")).pack(pady=10)

tk.Button(root, text="Selecionar Excel", width=30, command=selecionar_excel).pack()
lbl_excel = tk.Label(root, text="Nenhum arquivo selecionado")
lbl_excel.pack(pady=5)

frm = tk.Frame(root)
frm.pack(pady=10)

tk.Label(frm, text="Coluna inicial:").grid(row=0, column=0)
ent_inicio = tk.Entry(frm, width=6)
ent_inicio.insert(0, "2")
ent_inicio.grid(row=0, column=1)

tk.Label(frm, text="Coluna final:").grid(row=0, column=2)
ent_fim = tk.Entry(frm, width=6)
ent_fim.insert(0, "200")
ent_fim.grid(row=0, column=3)

chk_preco_var = tk.BooleanVar()
tk.Checkbutton(root, text="Inserir preço", variable=chk_preco_var).pack()

tk.Button(root, text="Iniciar", width=30, height=2, command=iniciar).pack(pady=4)
tk.Button(root, text="Cancelar", width=30, height=2, command=cancelar).pack(pady=4)

txt_log = ScrolledText(root, width=82, height=12)
txt_log.pack(pady=10)

tk.Button(root, text="Sair", width=20, command=root.destroy).pack()

root.mainloop()
